In [116]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iamsouravbanerjee/heart-attack-prediction-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/morning/.cache/kagglehub/datasets/iamsouravbanerjee/heart-attack-prediction-dataset/versions/2


In [117]:
import os
os.listdir(path)

['heart_attack_prediction_dataset.csv']

In [118]:
import pandas as pd
import fairtl as fl
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [119]:
df = pd.read_csv(f'{path}/heart_attack_prediction_dataset.csv')
df.head()

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,6.615001,261404,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,4.963459,285768,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,9.463426,235282,28.176571,587,4,4,France,Europe,Northern Hemisphere,0
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,7.648981,125640,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,1.514821,160555,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0


In [120]:
df[['systolic', 'diastolic']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)

In [121]:
df.head()

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk,systolic,diastolic
0,BMW7812,67,Male,208,158/88,72,0,0,1,0,...,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0,158,88
1,CZE1114,21,Male,389,165/93,98,1,1,1,1,...,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0,165,93
2,BNI9906,21,Female,324,174/99,72,1,0,0,0,...,28.176571,587,4,4,France,Europe,Northern Hemisphere,0,174,99
3,JLN3497,84,Male,383,163/100,73,1,1,1,0,...,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0,163,100
4,GFO8847,66,Male,318,91/88,93,1,1,1,1,...,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0,91,88


In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Patient ID                       8763 non-null   object 
 1   Age                              8763 non-null   int64  
 2   Sex                              8763 non-null   object 
 3   Cholesterol                      8763 non-null   int64  
 4   Blood Pressure                   8763 non-null   object 
 5   Heart Rate                       8763 non-null   int64  
 6   Diabetes                         8763 non-null   int64  
 7   Family History                   8763 non-null   int64  
 8   Smoking                          8763 non-null   int64  
 9   Obesity                          8763 non-null   int64  
 10  Alcohol Consumption              8763 non-null   int64  
 11  Exercise Hours Per Week          8763 non-null   float64
 12  Diet                

In [123]:
df['Blood Pressure'].unique()

array(['158/88', '165/93', '174/99', ..., '137/94', '94/76', '119/67'],
      shape=(3915,), dtype=object)

In [124]:
def classify_blood_pressure(bp):
    systolic, diastolic = map(int, bp.split('/'))
    
    if systolic < 120 and diastolic < 80:
        return 'Normal'
    elif 120 <= systolic <= 129 and diastolic < 80:
        return 'Elevated'
    elif 130 <= systolic <= 139 or 80 <= diastolic <= 89:
        return 'Hypertension Stage 1'
    elif 140 <= systolic <= 179 or 90 <= diastolic <= 119:
        return 'Hypertension Stage 2'
    elif systolic >= 180 or diastolic >= 120:
        return 'Hypertensive Crisis'
    else:
        return 'Unknown'

In [125]:
df['Blood Pressure'] = df['Blood Pressure'].apply(classify_blood_pressure)

In [126]:
df['Blood Pressure'].unique()

array(['Hypertension Stage 1', 'Hypertension Stage 2', 'Normal',
       'Elevated', 'Hypertensive Crisis'], dtype=object)

In [127]:
df['Blood Pressure'] = df['Blood Pressure'].map({'Normal': 0, 'Elevated': 1, 'Hypertension Stage 1': 2, 'Hypertension Stage 2': 3, 'Hypertensive Crisis': 4})

In [128]:
df.head()

,Patient ID,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,...,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk,systolic,diastolic
0,BMW7812,67,Male,208,2,72,0,0,1,0,...,31.251233,286,0,6,Argentina,South America,Southern Hemisphere,0,158,88
1,CZE1114,21,Male,389,3,98,1,1,1,1,...,27.194973,235,1,7,Canada,North America,Northern Hemisphere,0,165,93
2,BNI9906,21,Female,324,3,72,1,0,0,0,...,28.176571,587,4,4,France,Europe,Northern Hemisphere,0,174,99
3,JLN3497,84,Male,383,3,73,1,1,1,0,...,36.464704,378,3,4,Canada,North America,Northern Hemisphere,0,163,100
4,GFO8847,66,Male,318,2,93,1,1,1,1,...,21.809144,231,1,5,Thailand,Asia,Northern Hemisphere,0,91,88


In [129]:
df.drop(['Patient ID', 'Country', 'Continent', 'Hemisphere'], axis=1, inplace=True)
df['Sex'] = df['Sex'].map({'Female': 0, 'Male': 1})

In [130]:
df.head()

,Age,Sex,Cholesterol,Blood Pressure,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Stress Level,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Heart Attack Risk,systolic,diastolic
0,67,1,208,2,72,0,0,1,0,0,...,9,6.615001,261404,31.251233,286,0,6,0,158,88
1,21,1,389,3,98,1,1,1,1,1,...,1,4.963459,285768,27.194973,235,1,7,0,165,93
2,21,0,324,3,72,1,0,0,0,0,...,9,9.463426,235282,28.176571,587,4,4,0,174,99
3,84,1,383,3,73,1,1,1,0,1,...,9,7.648981,125640,36.464704,378,3,4,0,163,100
4,66,1,318,2,93,1,1,1,1,0,...,6,1.514821,160555,21.809144,231,1,5,0,91,88


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8763 entries, 0 to 8762
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              8763 non-null   int64  
 1   Sex                              8763 non-null   int64  
 2   Cholesterol                      8763 non-null   int64  
 3   Blood Pressure                   8763 non-null   int64  
 4   Heart Rate                       8763 non-null   int64  
 5   Diabetes                         8763 non-null   int64  
 6   Family History                   8763 non-null   int64  
 7   Smoking                          8763 non-null   int64  
 8   Obesity                          8763 non-null   int64  
 9   Alcohol Consumption              8763 non-null   int64  
 10  Exercise Hours Per Week          8763 non-null   float64
 11  Diet                             8763 non-null   object 
 12  Previous Heart Probl

In [132]:
df['Diet'] = df['Diet'].map({'Unhealthy': 0, 'Average': 1, 'Healthy': 2})

In [133]:
df.isna().sum().sum()

np.int64(0)

In [134]:
y = df['Heart Attack Risk'].values
X = df.drop('Heart Attack Risk', axis=1)

In [135]:
print(X.shape)
selector = VarianceThreshold(threshold=0.1)
X = selector.fit_transform(X)
print(X.shape)

(8763, 23)
(8763, 22)


In [136]:
# build mask
gender_0_mask = df['Sex'] == 0
gender_1_mask = df['Sex'] == 1

count_gender_0 = gender_0_mask.sum()
count_gender_1 = gender_1_mask.sum()

print("Female: ", count_gender_0)
print("Male: ", count_gender_1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_Gender_0 = X_scaled[gender_0_mask]
X_scaled_Gender_1 = X_scaled[gender_1_mask]
y_Gender_0 = y[gender_0_mask]
y_Gender_1 = y[gender_1_mask]

Female:  2652
Male:  6111


In [137]:
seed = 42
models = fl.build_models(seed)

results_list = []

kf = KFold(n_splits=10, shuffle=True, random_state=seed)

In [138]:
print("Starting experiments for Female(0)")
fl.run_experiment(kf, models, X_scaled_Gender_0, y_Gender_0, 'Female', results_list)

print("Starting experiments for Male(1)")
fl.run_experiment(kf, models, X_scaled_Gender_1, y_Gender_1, 'Male', results_list)

final_results_df = pd.concat(results_list, ignore_index=True)
print(final_results_df)

Starting experiments for Female(0)
Processing fold 1 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 2 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 3 for group Female
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


...

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 9 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN


/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training and evaluating model: NB
Processing fold 10 for group Male
Training and evaluating model: SVM
Training and evaluating model: LR
Training and evaluating model: KNN
Training and evaluating model: RF
Training and evaluating model: DT
Training and evaluating model: ANN
Training and evaluating model: NB
    Fold   Group   SVM_TPR   SVM_TNR   SVM_FPR   SVM_FNR  SVM_TP  SVM_TN  \
0      1  Female  0.000000  1.000000  0.000000  1.000000       0     180   
1      2  Female  0.000000  1.000000  0.000000  1.000000       0     174   
2      3  Female  0.000000  1.000000  0.000000  1.000000       0     160   
3      4  Female  0.000000  1.000000  0.000000  1.000000       0     160   
4      5  Female  0.000000  0.994048  0.005952  1.000000       0     167   
5      6  Female  0.000000  1.000000  0.000000  1.000000       0     166   
6      7  Female  0.000000  1.000000  0.000000  1.000000       0     168   
7      8  Female  0.000000  0.987879  0.012121  1.000000       0     163   
8      

/home/morning/miniconda3/envs/f2ml/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [139]:
results_df = pd.concat(results_list, ignore_index=True)

In [140]:
result_path = './results/k22_result.xlsx'
results_df.to_excel(result_path, index=False)

In [141]:
df = pd.read_excel(result_path)
df.head()

,Fold,Group,SVM_TPR,SVM_TNR,SVM_FPR,SVM_FNR,SVM_TP,SVM_TN,SVM_FP,SVM_FN,...,ANN_FP,ANN_FN,NB_TPR,NB_TNR,NB_FPR,NB_FNR,NB_TP,NB_TN,NB_FP,NB_FN
0,1,Female,0.0,1.000000,0.000000,1.0,0,180,0,86,...,46,70,0.034884,0.988889,0.011111,0.965116,3,178,2,83
1,2,Female,0.0,1.000000,0.000000,1.0,0,174,0,92,...,49,57,0.010870,0.971264,0.028736,0.989130,1,169,5,91
2,3,Female,0.0,1.000000,0.000000,1.0,0,160,0,105,...,46,69,0.019048,0.987500,0.012500,0.980952,2,158,2,103
3,4,Female,0.0,1.000000,0.000000,1.0,0,160,0,105,...,43,75,0.028571,0.987500,0.012500,0.971429,3,158,2,102
4,5,Female,0.0,0.994048,0.005952,1.0,0,167,1,97,...,39,77,0.010309,0.988095,0.011905,0.989691,1,166,2,96


In [142]:
label = 'Female'

fl.perform_t_tests(df, 'SVM', label)
fl.perform_t_tests(df, 'DT', label)
fl.perform_t_tests(df, 'RF', label)
fl.perform_t_tests(df, 'LR', label)
fl.perform_t_tests(df, 'KNN', label)
fl.perform_t_tests(df, 'ANN', label)
fl.perform_t_tests(df, 'NB', label)


SVM -TPR: MannwhitneyuResult(statistic=np.float64(41.5), pvalue=np.float64(0.38701255487244757))
SVM - FPR: MannwhitneyuResult(statistic=np.float64(61.5), pvalue=np.float64(0.23426628444669173))
SVM - FN/FP: MannwhitneyuResult(statistic=np.float64(7.0), pvalue=np.float64(0.001303916782021204))
DT -TPR: TtestResult(statistic=np.float64(0.24469407695966539), pvalue=np.float64(0.8094610366165043), df=np.float64(18.0))
DT - FPR: TtestResult(statistic=np.float64(0.8853893424470205), pvalue=np.float64(0.3876218497516424), df=np.float64(18.0))
DT - FN/FP: TtestResult(statistic=np.float64(-0.3477908567206795), pvalue=np.float64(0.7320311928407215), df=np.float64(18.0))
RF -TPR: TtestResult(statistic=np.float64(2.880496852660801), pvalue=np.float64(0.009955753091761175), df=np.float64(18.0))
RF - FPR: TtestResult(statistic=np.float64(2.8359945855501176), pvalue=np.float64(0.010956811479202515), df=np.float64(18.0))
RF - FN/FP: MannwhitneyuResult(statistic=np.float64(17.0), pvalue=np.float64(0.0